In [1]:
import pandas as pd

# df = pd.read_csv('use_this_data/rest_data_last.csv')
# df = pd.read_csv('data_2.csv')
df = pd.read_csv('use_this_data/data_features.csv')
df.head()

,category,text,clean_text,matches,features
0,arts,rob delaney vir das galen hopper samson kayo g...,rob hopper samson kayo guz khan nick ross lee ...,True,rob hopper samson kayo guz nick ross lee harry...
1,arts,andris nelsons conducts a joint concert of the...,nelson conduct joint concert boston symphony o...,True,nelson conduct joint concert boston symphony o...
2,arts,warner music group has brought on sherry tan t...,warner music group brought sherry tan head mus...,True,warner music group brought sherry tan head mus...
3,arts,adele will explore what she s been going throu...,explore going album set explore going album ol...,True,explore going album set explore going album ol...
4,arts,you are using an older browser version. please...,older browser version please use version best ...,True,browser version please use version experience ...


In [2]:
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from nltk import sent_tokenize
import nltk
from collections import Counter
import nltk
from collections import Counter
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier


import copy

weighted_dict= {}

new_keywords = {
    'arts': [
        'music','show','series','park','best','million','event','back','work','state',
        'way','may','get','group','take','world','community','season','good','school'
    ],
    'crime': [
        'police','court','law','state','case','county','old','found','may','hearing',
        'death','federal','prison','told','justice','man','right','officer','according','president'
    ],
    'disaster': [
        'fire','water','police','air','state','rescue','city','may','area','county',
        'home','near','according','news','around','could','back','national','help','road'
    ],
    'economy': [
        'company','may','per','market','million','price','tax','billion','business','government',
        'state','since','industry','group','percent','use','get','made','work','according'
    ],
    'education': [
        'school','university','education','high','learning','college','state','student','class',
        'community','work','program','support','help','board','government','president','covid','public','national'
    ],
    'environmental': [
        'energy','park','waste','water','power','market','state','company','north','management',
        'solar','group','renewable','county','world','wildlife','city','wind','river','industry'
    ],
    'health': [
        'health','covid','care','market','medical','pandemic','hospital','state','may','vaccine',
        'public','food','disease','get','help','company','global','well','virus','work'
    ],
    'humaninterest': [
        'award','size','get','act','home','back','winning','best','statement','texas',
        'team','may','good','world','well','ownership','beneficial','plant','second','top'
    ],
    'labour': [
        'work','health','security','working','pandemic','retirement','may','covid','could','get',
        'state','make','job','business','home','help','social','market','city','week'
    ],
    'lifestyle': [
        'home','travel','get','beauty','bridge','make','summer','may','way','life',
        'work','back','million','come','food','covid','want','pandemic','best','business'
    ],
    # 'other': [
    #     'state','market','game','covid','government','may','could','rub','wednesday','home',
    #     'high','city','get','school','pandemic','health','make','season','back','team'
    # ],
    'other': [
        'may','could','rub','wednesday','home',
        'high','city','get','school','health','make','season','back','team'
    ],
    'politics': [
        'government','state','president','minister','data','security','public','city','international','support',
        'tuesday','news','united','may','covid','national','pandemic','help','house','health'
    ],
    'religion': [
        'church','life','school','family','may','community','world','way','come','work',
        'home','covid','old','get','back','spiritual','god','state','well','city'
    ],
    'science': [
        'market','research','space','report','global','data','work','company','study','industry',
        'may','science','medical','could','high','design','engineering','covid','well','use'
    ],
    'social': [
        'health','care','social','work','government','covid','may','baby','state','abortion',
        'support','home','get','life','pandemic','according','community','make','city','help'
    ],
    'sport': [
        'team','game','season','back','world','get','match','second','sport','four',
        'league','win','best','final','play','tournament','coach','right','open','championship'
    ],
    'unrest': [
        'israel','police','war','government','city','violence','shooting','israeli','military','may',
        'wednesday','state','gun','country','president','old','come','security','world','home'
    ],
    'weather': [
        'climate','change','heat','weather','water','high','carbon','could','global','national',
        'state','may','risk','area','service','according','world','friday','rain','report'
    ],
}



# 3) Create a copy so we don't modify original directly
updated_dict = copy.deepcopy(weighted_dict)

# 4) Define a default weight for new words
DEFAULT_WEIGHT = 0.6

# 5) Merge logic
for category, words in new_keywords.items():
    # If this category doesn't exist in the dictionary, create it
    if category not in updated_dict:
        updated_dict[category] = {}
    
    for w in words:
        # Add the word only if it isn't already in the dictionary
        if w not in updated_dict[category]:
            updated_dict[category][w] = DEFAULT_WEIGHT

# 'updated_dict' is now the merged result.


x = df["clean_text"]
y = df["category"]
# Train the Random Forest classifier
X_train, X_test, y_train, y_test = train_test_split(
    x, y, test_size=0.2, random_state=42
)

vectorizer = TfidfVectorizer()
X_train_vectorized = vectorizer.fit_transform(X_train)
X_test_vectorized = vectorizer.transform(X_test)

rf = RandomForestClassifier(n_estimators=100, random_state=42)
rf.fit(X_train_vectorized, y_train)

y_pred = rf.predict(X_test_vectorized)

accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

def dictionary_score(sentence, weighted_dict):
    """
    Computes a score for each class based on the presence of weighted keywords.
    """
    words = sentence.lower().split()
    scores = {cls: 0 for cls in weighted_dict}
    for cls in weighted_dict:
        for word in words:
            scores[cls] += weighted_dict[cls].get(word, 0)
    return scores

def classifier_predict(sentence):
    X_sentence = vectorizer.transform([sentence])
    probas = rf.predict_proba(X_sentence)[0]
    return dict(zip(rf.classes_, probas))

def combine_predictions(clf_pred, dict_scores, alpha=0.5):
    """
    Combine classifier probabilities and dictionary scores.
    The parameter alpha weights the classifier output.
    """
    combined = {}
    for cls in clf_pred:
        combined[cls] = alpha * clf_pred[cls] + (1 - alpha) * dict_scores.get(cls, 0)
    return combined

def classify_text(text, alpha=0.5):
    """
    Tokenize the text into sentences, combine classifier and dictionary scores,
    and return the final predicted class based on majority vote.
    """
    sentences = sent_tokenize(text)
    sentence_labels = []

    for sentence in sentences:
        clf_pred = classifier_predict(sentence)
        dict_scores = dictionary_score(sentence, weighted_dict)
        combined_scores = combine_predictions(clf_pred, dict_scores, alpha)
        # Select the class with the highest combined score for the sentence
        sentence_label = max(combined_scores, key=combined_scores.get)
        sentence_labels.append(sentence_label)

    # Final classification by taking the most common class (majority vote)
    final_label = Counter(sentence_labels).most_common(1)[0][0]
    return final_label

# Example usage:
text_row = ("The gallery showcased a variety of paintings and sculptures. "
            "A recent experiment in the laboratory produced groundbreaking results.")
final_prediction = classify_text(text_row, alpha=0.65)
print("Final predicted class:", final_prediction)

Accuracy: 0.5771643663739021
Final predicted class: economy


In [3]:
def dictionary_score(sentence, weighted_dict):
    """
    Computes a score for each class based on the presence of weighted keywords.
    """
    words = sentence.lower().split()
    scores = {cls: 0 for cls in weighted_dict}
    for cls in weighted_dict:
        for word in words:
            scores[cls] += weighted_dict[cls].get(word, 0)
    return scores

def classifier_predict(sentence):
    """
    Uses the trained Random Forest and TF-IDF vectorizer to get class probabilities.
    """
    X_sentence = vectorizer.transform([sentence])
    probas = rf.predict_proba(X_sentence)[0]
    return dict(zip(rf.classes_, probas))

def combine_predictions(clf_pred, dict_scores, alpha=0.5):
    """
    Combine classifier probabilities and dictionary scores.
    The parameter alpha weights the classifier output.
    """
    combined = {}
    for cls in clf_pred:
        combined[cls] = alpha * clf_pred[cls] + (1 - alpha) * dict_scores.get(cls, 0)
    return combined

def classify_text(text, alpha=0.5):
    """
    Tokenize the text into sentences, combine classifier and dictionary scores,
    and return the final predicted class based on majority vote.
    """
    sentences = sent_tokenize(text)
    sentence_labels = []
    
    for sentence in sentences:
        clf_pred = classifier_predict(sentence)
        dict_scores = dictionary_score(sentence, weighted_dict)
        combined_scores = combine_predictions(clf_pred, dict_scores, alpha)
        
        # Select the class with the highest combined score for the sentence
        sentence_label = max(combined_scores, key=combined_scores.get)
        sentence_labels.append(sentence_label)
    
    # Final classification by taking the most common class (majority vote)
    final_label = Counter(sentence_labels).most_common(1)[0][0]
    return final_label

from tqdm import tqdm
tqdm.pandas()  # Enable progress_apply on pandas objects
import numpy as np

df["predicted_category"] = df["clean_text"].progress_apply(lambda x: classify_text(x, alpha=0.65))

df.head()


100%|██████████| 3985/3985 [00:10<00:00, 383.52it/s]


,category,text,clean_text,matches,features,predicted_category
0,arts,rob delaney vir das galen hopper samson kayo g...,rob hopper samson kayo guz khan nick ross lee ...,True,rob hopper samson kayo guz nick ross lee harry...,arts
1,arts,andris nelsons conducts a joint concert of the...,nelson conduct joint concert boston symphony o...,True,nelson conduct joint concert boston symphony o...,arts
2,arts,warner music group has brought on sherry tan t...,warner music group brought sherry tan head mus...,True,warner music group brought sherry tan head mus...,arts
3,arts,adele will explore what she s been going throu...,explore going album set explore going album ol...,True,explore going album set explore going album ol...,arts
4,arts,you are using an older browser version. please...,older browser version please use version best ...,True,browser version please use version experience ...,arts


In [4]:
df['predicted_category'].value_counts()

predicted_category
weather          286
education        285
sport            272
religion         260
labour           257
health           244
politics         243
science          235
disaster         228
environmental    221
crime            205
lifestyle        199
unrest           199
arts             197
economy          180
humanInterest    177
social           169
other            128
Name: count, dtype: int64

In [5]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report

# Example DataFrame: df with columns "text2" and "total_category"
x = df["clean_text"]
y = df["predicted_category"]

# 1. Train-test split
X_train, X_test, y_train, y_test = train_test_split(
    x, y, test_size=0.2, random_state=42
)

# 2. TF-IDF vectorization
vectorizer = TfidfVectorizer(
    analyzer='word',
    token_pattern=r'(?u)\b\w\w+\b',
    lowercase=True,
    stop_words='english',
    ngram_range=(1, 2),
    min_df=2,
    max_df=0.8,
    # max_features=50000,
    sublinear_tf=True,
    norm='l2',
    smooth_idf=True
)
X_train_vectorized = vectorizer.fit_transform(X_train)
X_test_vectorized = vectorizer.transform(X_test)

# 3. Define parameter distributions for RandomizedSearchCV
param_distributions = {
    "n_estimators": [100, 200, 300, 400, 500],
    "max_depth": [5, 10, 15, 20, None],
    "min_samples_split": [2, 5, 10],
    "min_samples_leaf": [1, 2, 4],
    "bootstrap": [True, False],
    "criterion": ["gini", "entropy"],
    "class_weight": ["balanced", None, "balanced_subsample"],
}

# 4. Base RandomForest model
rf = RandomForestClassifier(random_state=42)

# 5. RandomizedSearchCV setup
random_search = RandomizedSearchCV(
    estimator=rf,
    param_distributions=param_distributions,
    n_iter=30,
    cv=3,
    scoring='accuracy',
    random_state=42,
    verbose=1,
    n_jobs=-1
)

# 6. Fit random search on training data
random_search.fit(X_train_vectorized, y_train)

# 7. Get the best model and hyperparameters
best_rf = random_search.best_estimator_
print("Best Hyperparameters found by RandomizedSearchCV:")
print(random_search.best_params_)

# 8. Predict on test data (standard prediction, no custom threshold)
y_pred = best_rf.predict(X_test_vectorized)

# 9. Evaluate
accuracy = accuracy_score(y_test, y_pred)
print(f"\nAccuracy: {accuracy}")
print("\nClassification Report:")
print(classification_report(y_test, y_pred))


Fitting 3 folds for each of 30 candidates, totalling 90 fits
Best Hyperparameters found by RandomizedSearchCV:
{'n_estimators': 400, 'min_samples_split': 10, 'min_samples_leaf': 4, 'max_depth': None, 'criterion': 'gini', 'class_weight': 'balanced_subsample', 'bootstrap': True}

Accuracy: 0.8105395232120451

Classification Report:
               precision    recall  f1-score   support

         arts       0.77      0.93      0.84        40
        crime       0.75      0.90      0.81        49
     disaster       0.79      0.71      0.75        42
      economy       0.74      0.77      0.75        30
    education       0.94      0.90      0.92        81
environmental       0.84      0.92      0.88        50
       health       0.68      0.84      0.75        55
humanInterest       0.78      0.83      0.81        30
       labour       0.86      0.80      0.83        60
    lifestyle       0.57      0.57      0.57        30
        other       0.00      0.00      0.00         1
     po

/Users/karennurlybekov/miniconda3/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/karennurlybekov/miniconda3/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/karennurlybekov/miniconda3/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()

In [6]:
def predict_class(text, model, tfidf_vectorizer):
    """
    Given a single text string, this function returns the predicted class.
    """
    text_vectorized = tfidf_vectorizer.transform([text])
    predicted_label = model.predict(text_vectorized)[0]
    return predicted_label

df_validation = pd.read_csv("data/validation_data.csv")

X_validation_vectorized = vectorizer.transform(df_validation["text"])
val_predictions = best_rf.predict(X_validation_vectorized)
df_validation["predicted_category"] = val_predictions

df_validation.to_csv("results/validation_predictions.csv", index=False)

print("Validation Predictions:")
print(df_validation)

Validation Predictions:
    category                                               text  \
0       arts  you are using an older browser version. please...   
1       arts  on 31 march two of classical music s most acco...   
2       arts  bpt after a year of being locked away at home ...   
3       arts  pilot uninjured plane hit sandbar while landin...   
4       arts  colleen distin photo by facebook toronto sun ....   
..       ...                                                ...   
103  weather  snow measurements from june 1 varied widely ar...   
104  weather  province recognizes world environment day . pr...   
105  weather  seattle prnewswire rei co op is investing over...   
106  weather  june 1 marked the official start of the atlant...   
107  weather  covering up with a hat to ward off heat exhaus...   

    predicted_category  
0            lifestyle  
1                 arts  
2                 arts  
3             disaster  
4                 arts  
..                 ..

In [13]:
def predict_text(text, model, tfidf_vectorizer):
    """
    Given a single text string, this function returns the predicted class.
    :param text: 
    :param model: 
    :param tfidf_vectorizer: 
    :return: 
    """
    text_vectorized = tfidf_vectorizer.transform([text])
    predicted_label = model.predict(text_vectorized)[0]
    return predicted_label

messege = input("Enter a text: ")
X_values = vectorizer.transform([messege])
single_prediction = predict_text(messege, best_rf, vectorizer)


print("Validation Predictions:")
print(single_prediction)

Validation Predictions:
disaster


In [12]:
import joblib

joblib.dump(rf, "./random_forest.joblib")

['./random_forest.joblib']